In [ ]:
import praw
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# --- Sentiment Analysis Setup ---
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

# Map sentiment labels
sentiment_label_map = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

# --- Emotion Detection Setup ---
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_pipeline = pipeline("text-classification", model=emotion_model_name, tokenizer=emotion_model_name, top_k=1)

# --- Reddit Authentication ---
reddit = praw.Reddit(
    client_id='q_dYyqYYdNNInGsM-lC9Xg',
    client_secret='pLigWA6vX6llH7NjWBhVWmg-gJjKvg',
    user_agent='script:gaming_trend (by /u/HiGhastlyy)',
    username='HiGhastlyy',
    password='0306terror'
)

# --- Scraping Posts ---
subreddits = ['gaming', 'pcgaming', 'leagueoflegends', 'cs2', 'dota2', 'overwatch', 'fortnite', 'apexlegends', 'valorant', 'minecraft']
posts = []

for sub in subreddits:
    for post in reddit.subreddit(sub).top(time_filter='year', limit=500):
        full_text = f"{post.title} {post.selftext}"

        # Sentiment prediction
        sentiment_result = sentiment_pipeline(full_text[:512])[0]  # truncate to 512 tokens max
        sentiment = sentiment_label_map[sentiment_result['label']]
        sentiment_score = sentiment_result['score']

        # Emotion prediction
        emotion_result = emotion_pipeline(full_text[:512])[0]
        emotion = emotion_result['label']
        emotion_score = emotion_result['score']

        posts.append({
            'id': post.id,
            'subreddit': sub,
            'title': post.title,
            'body': post.selftext,
            'score': post.score,
            'num_comments': post.num_comments,
            'created_utc': post.created_utc,
            'sentiment': sentiment,
            'sentiment_confidence': round(sentiment_score, 3),
            'emotion': emotion,
            'emotion_confidence': round(emotion_score, 3)
        })

# --- Save to CSV ---
df = pd.DataFrame(posts)
df.to_csv("reddit_gaming_trends.csv", index=False)

print("Done! Data saved to reddit_gaming_trends.csv")

Done! Data saved to reddit_gaming_trends.csv
